In [1]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd
import fiona
import shapely

from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

from utilities import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [41]:
major_bus = gpd.read_parquet(f'{GCS_FILE_PATH}major_bus_stops.parquet')
major_bus['mode'] = 'bus'

In [42]:
major_bus.head(3)

,calitp_itp_id,stop_id,geometry,hqta_type,mode
0,142,3977,POINT (199058.382 -471125.032),major_transit_stop,bus
1,142,3977,POINT (198531.592 -471368.189),major_transit_stop,bus
2,142,3977,POINT (193068.693 -465847.391),major_transit_stop,bus


In [ ]:
## rail/ferry/brt explicitly by mode

In [12]:
rail_ferry_brt_stops = gpd.read_parquet(f'{GCS_FILE_PATH}rail_brt_ferry.parquet')

In [43]:
rail_ferry_brt_stops.columns

Index(['stop_id', 'calitp_itp_id', 'stop_lat', 'stop_lon', 'itp_id_stop_id',
       'itp_id_trip_id', 'route_id', 'itp_id_route_id', 'trip_id', 'geometry',
       'calitp_url_number', 'route_type', 'agency_id', 'route_short_name',
       'route_long_name', 'route_desc', 'route_url', 'route_color',
       'route_text_color', 'route_sort_order', 'continuous_pickup',
       'continuous_drop_off', 'calitp_extracted_at'],
      dtype='object')

In [46]:
rail_ferry_brt_stops['route_type'] = rail_ferry_brt_stops['route_type'].fillna('99')

In [47]:
rail_ferry_brt_stops >> count(_.calitp_itp_id, _.route_type)

,calitp_itp_id,route_type,n
0,2,4,14
1,4,3,67
2,10,99,10
3,13,99,523
4,15,4,5
5,41,4,5
6,56,99,18
7,127,4,4
8,182,3,55
9,182,99,95


In [44]:
rail_ferry_brt_stops[['stop_id', 'calitp_itp_id', 'route_type']]

,stop_id,calitp_itp_id,route_type
0,CRN,13,None
1,MKA,13,None
2,OKJ,13,None
3,PLO,13,None
4,QCY,13,None
...,...,...,...
74,2483569,41,4
77,GF:43000,200,4
78,GF:43003,200,4
79,GF:43004,200,4


In [ ]:
## other stops within hq bus corridor (mode=bus)

In [2]:
bus_hqtc = gpd.read_parquet(f'{GCS_FILE_PATH}shape_hqta_dissolve.parquet')
bus_hqtc = bus_hqtc[bus_hqtc['hq_transit_corr']]
bus_hqtc['hqta_type'] = 'hq_transit_corr'

In [30]:
all_stops = tbl.gtfs_schedule.stops() >> collect()
all_stops = gpd.GeoDataFrame(all_stops,
                             geometry = gpd.points_from_xy(all_stops.stop_lon, all_stops.stop_lat),
                            crs = 'EPSG:4326')